In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import gaitFunctions
import plotClip

In [26]:
# navigate to directory that contains the data
os.chdir('/Users/iwoods/OneDrive - Ithaca College/movement_disorder_paper_2023/videos_exemplaris/analyses_summaries/')
for f in sorted(glob.glob('*.xlsx')):
    print(f)

all_exemplaris_through_9Nov23_combined.xlsx
iw_9Nov23_exemplaris_inverted_combined.xlsx


In [23]:
# load the step timing data into a dataframe
data_file = 'iw_9Nov23_exemplaris_inverted_combined.xlsx'
step_df = pd.read_excel(data_file, sheet_name='step_timing', index_col=None)
step_df.head(3)

,legID,DownTime,UpTime,stance,swing,gait,duty,midSwingTime,L1_mid_swings,R1_mid_swings,...,average_tardigrade_area,average_tardigrade_length,anterior_offsets,contralateral_offsets,metachronal_lag,clip,treatment,individual,date,uniq_id
0,L1,0.363,1.575,1.212,0.242,1.454,0.8336,1.6960,L1:0.9168,R1:,...,5571.166667,149.225283,NaN,0.212,NaN,iw_9Nov_tardigrade01_exemplaris_033-060,exemplaris,1,9nov,iw_9nov_tardigrade01_exemplaris
1,L1,1.817,2.635,0.818,0.242,1.060,0.7717,2.7560,L1:0.8858,R1:0.1,...,5594.528571,148.743342,NaN,0.363,NaN,iw_9Nov_tardigrade01_exemplaris_033-060,exemplaris,1,9nov,iw_9nov_tardigrade01_exemplaris
2,L1,2.877,3.634,0.757,0.243,1.000,0.7570,3.7555,L1:0.8785,R1:0.2425,...,5672.060606,150.775930,NaN,0.455,NaN,iw_9Nov_tardigrade01_exemplaris_033-060,exemplaris,1,9nov,iw_9nov_tardigrade01_exemplaris


,Identifier,treatment,individual,Stance duration (lateral legs),Swing duration (lateral legs),Gait cycle (lateral legs),Duty factor (lateral legs),mm per step (lateral legs),bodylength per step (lateral legs),Stance duration (rear legs),...,bodylength per step (rear legs),Metachronal lag (lateral legs),"Metachronal lag (normalized, lateral legs)",Metachronal lag Left-Right Ratio,Anterior swing offsets (lateral legs),"Anterior swing offsets (normalized, lateral legs)",Opposite swing offsets (lateral legs),"Opposite swing offsets (normalized, lateral legs)",Opposite swing offsets (rear legs),"Opposite swing offsets (normalized, rear legs)"
0,iw_9nov_tardigrade01_exemplaris,exemplaris,tardigrade01,0.851489,0.275311,1.126800,0.748824,0.029142,0.189510,0.617375,...,0.178503,1.309538,1.162175,0.448615,0.659069,0.584903,0.550000,0.488108,0.469625,0.442990
1,iw_9nov_tardigrade02_exemplaris,exemplaris,tardigrade02,0.475855,0.254179,0.730034,0.648824,0.077522,0.305536,0.241000,...,0.217514,0.551229,0.754712,0.090724,0.275405,0.377058,0.372782,0.512526,0.243704,0.466708
2,iw_9nov_tardigrade03_exemplaris,exemplaris,tardigrade03,0.534930,0.239296,0.774225,0.690037,0.048334,0.224796,0.391043,...,0.224677,0.488727,0.631247,0.041330,0.243630,0.314676,0.391029,0.505059,0.346636,0.446922
